In [1]:
import pandas as pd
import numpy as np
import anndata
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings
import pathlib

warnings.filterwarnings('ignore')

In [2]:
from matplotlib import rc
labelsize = 6
linewidth = 0.6
rc('lines', linewidth=linewidth)
rc('axes', labelsize=labelsize, linewidth=linewidth)
rc('xtick', labelsize=labelsize)
rc('ytick', labelsize=labelsize)
rc('xtick.major', width=linewidth)
rc('ytick.major', width=linewidth)
rc('xtick.minor', width=linewidth - 0.2)
rc('ytick.minor', width=linewidth - 0.2)

In [3]:
chrom = 'chr2'
genes = ['Lhx6']
slop = 250000
n_pc = 10
resolution = 1

In [4]:
# Parameters
chrom = "chr19"
genes = ["ENSMUSG00000033768.17", "ENSMUSG00000024642.17", "ENSMUSG00000025020.11", "ENSMUSG00000025089.15", "ENSMUSG00000024897.9", "ENSMUSG00000025085.16", "ENSMUSG00000039901.10", "ENSMUSG00000040929.17", "ENSMUSG00000041362.17", "ENSMUSG00000048720.8", "ENSMUSG00000117897.1", "ENSMUSG00000024713.16", "ENSMUSG00000024867.14", "ENSMUSG00000041731.13", "ENSMUSG00000052942.13", "ENSMUSG00000024743.16", "ENSMUSG00000006435.16", "ENSMUSG00000033610.16", "ENSMUSG00000024798.15", "ENSMUSG00000058624.13", "ENSMUSG00000071604.5", "ENSMUSG00000039982.8", "ENSMUSG00000097930.2", "ENSMUSG00000025092.7", "ENSMUSG00000074746.3", "ENSMUSG00000024921.17", "ENSMUSG00000036278.7", "ENSMUSG00000024816.12", "ENSMUSG00000053617.12", "ENSMUSG00000025025.14", "ENSMUSG00000025189.9", "ENSMUSG00000074733.14", "ENSMUSG00000039126.10", "ENSMUSG00000033053.4", "ENSMUSG00000024754.13", "ENSMUSG00000009378.4", "ENSMUSG00000024781.16", "ENSMUSG00000025026.15", "ENSMUSG00000025090.9", "ENSMUSG00000024812.11", "ENSMUSG00000032946.16", "ENSMUSG00000024805.16", "ENSMUSG00000036192.15", "ENSMUSG00000024924.14", "ENSMUSG00000024983.10", "ENSMUSG00000035171.9", "ENSMUSG00000046324.12", "ENSMUSG00000025221.15", "ENSMUSG00000024935.11", "ENSMUSG00000025171.2", "ENSMUSG00000067577.8", "ENSMUSG00000067297.5", "ENSMUSG00000064105.13", "ENSMUSG00000087002.1", "ENSMUSG00000025195.9", "ENSMUSG00000069833.13", "ENSMUSG00000038658.15", "ENSMUSG00000114721.1", "ENSMUSG00000025231.17", "ENSMUSG00000024795.11", "ENSMUSG00000025217.15", "ENSMUSG00000024776.17", "ENSMUSG00000034765.6", "ENSMUSG00000057858.8", "ENSMUSG00000035818.14", "ENSMUSG00000094732.2", "ENSMUSG00000024677.13", "ENSMUSG00000024913.16", "ENSMUSG00000025078.9", "ENSMUSG00000033417.16", "ENSMUSG00000074925.4", "ENSMUSG00000032773.9", "ENSMUSG00000053279.8", "ENSMUSG00000024978.11", "ENSMUSG00000047787.8", "ENSMUSG00000041180.13", "ENSMUSG00000052085.7", "ENSMUSG00000058470.10", "ENSMUSG00000024727.9", "ENSMUSG00000024670.17", "ENSMUSG00000084825.1", "ENSMUSG00000035735.10", "ENSMUSG00000025076.10", "ENSMUSG00000025041.17", "ENSMUSG00000024729.7", "ENSMUSG00000097636.8", "ENSMUSG00000024785.7", "ENSMUSG00000118340.1", "ENSMUSG00000024639.5", "ENSMUSG00000024665.8", "ENSMUSG00000024678.6", "ENSMUSG00000025016.11", "ENSMUSG00000024697.4", "ENSMUSG00000042532.14", "ENSMUSG00000064202.15", "ENSMUSG00000071573.14", "ENSMUSG00000094707.2", "ENSMUSG00000046585.9", "ENSMUSG00000118150.1", "ENSMUSG00000035173.14", "ENSMUSG00000118020.1", "ENSMUSG00000025086.13", "ENSMUSG00000033863.2", "ENSMUSG00000052188.6", "ENSMUSG00000042694.17", "ENSMUSG00000024855.10", "ENSMUSG00000061742.12", "ENSMUSG00000035283.5", "ENSMUSG00000026931.10", "ENSMUSG00000110007.2", "ENSMUSG00000057240.14", "ENSMUSG00000086213.2", "ENSMUSG00000100417.2", "ENSMUSG00000049303.10", "ENSMUSG00000024786.9", "ENSMUSG00000024908.15", "ENSMUSG00000048612.16", "ENSMUSG00000110298.1", "ENSMUSG00000040022.14", "ENSMUSG00000118127.1", "ENSMUSG00000024837.15", "ENSMUSG00000118212.1", "ENSMUSG00000024664.5", "ENSMUSG00000025017.10", "ENSMUSG00000084876.7", "ENSMUSG00000038843.18", "ENSMUSG00000047731.17", "ENSMUSG00000024941.9", "ENSMUSG00000025002.5", "ENSMUSG00000024992.13", "ENSMUSG00000094584.2", "ENSMUSG00000063683.12", "ENSMUSG00000117990.1", "ENSMUSG00000024976.14", "ENSMUSG00000079419.4", "ENSMUSG00000035179.4", "ENSMUSG00000024974.10", "ENSMUSG00000048120.16", "ENSMUSG00000025077.13", "ENSMUSG00000046650.4", "ENSMUSG00000024654.8", "ENSMUSG00000117871.1", "ENSMUSG00000032702.16", "ENSMUSG00000024669.8", "ENSMUSG00000036961.5", "ENSMUSG00000056290.16", "ENSMUSG00000097740.2", "ENSMUSG00000052595.8", "ENSMUSG00000039652.16", "ENSMUSG00000025083.18", "ENSMUSG00000118293.1", "ENSMUSG00000060675.13", "ENSMUSG00000041360.8", "ENSMUSG00000117956.1", "ENSMUSG00000117655.1", "ENSMUSG00000048029.10", "ENSMUSG00000100001.1", "ENSMUSG00000099757.1", "ENSMUSG00000024937.15", "ENSMUSG00000045932.12", "ENSMUSG00000118109.1", "ENSMUSG00000024757.7", "ENSMUSG00000033033.10", "ENSMUSG00000117728.1", "ENSMUSG00000117981.1", "ENSMUSG00000086121.1", "ENSMUSG00000047368.4", "ENSMUSG00000025035.9", "ENSMUSG00000024681.11", "ENSMUSG00000040565.8", "ENSMUSG00000118273.1", "ENSMUSG00000025207.16", "ENSMUSG00000024811.11", "ENSMUSG00000024789.13", "ENSMUSG00000025200.7", "ENSMUSG00000024985.19", "ENSMUSG00000024952.13", "ENSMUSG00000067242.11", "ENSMUSG00000024691.13", "ENSMUSG00000117690.1", "ENSMUSG00000025082.8", "ENSMUSG00000117914.1", "ENSMUSG00000025091.4", "ENSMUSG00000059363.10", "ENSMUSG00000025007.13", "ENSMUSG00000024780.7", "ENSMUSG00000024900.5", "ENSMUSG00000025154.15", "ENSMUSG00000025047.9", "ENSMUSG00000101084.1", "ENSMUSG00000025094.8", "ENSMUSG00000101389.1", "ENSMUSG00000067656.13", "ENSMUSG00000024800.8", "ENSMUSG00000071633.11", "ENSMUSG00000061132.13", "ENSMUSG00000025184.10", "ENSMUSG00000117945.1", "ENSMUSG00000060049.5", "ENSMUSG00000034459.8", "ENSMUSG00000117998.1", "ENSMUSG00000043969.4", "ENSMUSG00000063796.9", "ENSMUSG00000101088.6", "ENSMUSG00000065594.1", "ENSMUSG00000024682.3", "ENSMUSG00000025197.9", "ENSMUSG00000097735.7", "ENSMUSG00000024943.9"]
slop = 250000
n_pc = 10
resolution = 1


In [5]:
output_dir = pathlib.Path(chrom)
output_dir.mkdir(exist_ok=True)

## Cell Meta

In [6]:
cell_tidy_data = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
)
use_clusters = [
    i.replace(' ', '_') for i in cell_tidy_data[cell_tidy_data['CellClass'].isin(['Exc', 'Inh'])]
    ['SubType'].unique() if 'Outlier' not in i
]
len(use_clusters)

145

## ATAC peaks

In [7]:
atac_peak = pd.read_msgpack('/home/hanliu/project/mouse_rostral_brain/study/DMRCluster/SubType.ATAC_peak_merged.msg')
atac_peak = atac_peak.loc[atac_peak.index.map(lambda i: i.startswith(f'Sub{chrom}_'))].copy()

## Gene Info

In [8]:
gene_meta = pd.read_csv(
    '/home/hanliu/ref/mouse/gencode/vm22/gencode.vM22.annotation.gene.flat.tsv.gz',
    index_col='gene_id', sep='\t'
)
gene_meta = gene_meta[gene_meta['chrom'] == chrom].copy()

In [9]:
exon_bed = pd.read_csv('/home/hanliu/ref/mouse/gencode/vm22/genome_anno/exon.all.bed',
                       header=None, sep='\t')
exon_bed.columns = ['chrom', 'start', 'end', 'gene_id', 'gene_name']

## DMR Info

In [10]:
with pd.HDFStore('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRInfo.h5', 'r') as hdf:
    dmr_rate = hdf['Rate']
dmr_rate = dmr_rate.loc[dmr_rate.index.map(lambda i: i.startswith(f'Sub{chrom}_'))].copy()

dmr_corr = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/DMRGeneCorr/TotalGeneDMRCorrLoop.0.3.msg'
)
dmr_corr = dmr_corr.set_index(['DMR', 'Gene'])
dmr_corr = dmr_corr.loc[dmr_corr.index.get_level_values('DMR').isin(dmr_rate.index)].copy()

dmr_bed = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalDMR.nofilter.bed',
                      sep='\t', header=None, index_col=3)
dmr_bed.columns = ['chrom', 'start', 'end']
dmr_bed = dmr_bed[dmr_bed['chrom'] == chrom].copy()

dmr_hits = anndata.read_h5ad('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalHits.HypoDMR.h5ad')
dmr_hits = dmr_hits[dmr_rate.index, :].copy()
dmr_hits = dmr_hits[:, use_clusters].copy()

dmr_annot = anndata.read_h5ad('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRAnnotation.h5ad')
dmr_annot = dmr_annot[dmr_rate.index, :].copy()

In [11]:
dmr_hits.shape[0]

104098

In [12]:
def get_gene(gene_id):
    _gene = gene_meta.loc[gene_id]
    return _gene.name, _gene['chrom'], _gene['start'], _gene['end'], _gene['strand']

## Gene's DMR clustering

In [13]:
def calculate_gene(gene_id):
    # get gene information
    gene_id, _, gene_start, gene_end, _ = get_gene(gene_id)

    # select related DMRs
    related_dmr = dmr_bed[(dmr_bed['start'] > gene_start - slop) &
                          (dmr_bed['end'] < gene_end + slop)].copy()
    
    related_dmr_rate = dmr_rate.loc[related_dmr.index, use_clusters].copy()
    related_dmr_rate.fillna(related_dmr_rate.mean(), axis=0, inplace=True)
    
    # construct Adata
    adata = anndata.AnnData(X=related_dmr_rate.values.copy(),
                            obs=pd.DataFrame([], related_dmr_rate.index),
                            var=pd.DataFrame([], related_dmr_rate.columns))
    sc.pp.scale(adata)
    sc.pp.pca(adata)
    
    pcs = adata.obsm['X_pca'][:, :n_pc]
    sc.pp.neighbors(adata, n_neighbors=int(round(np.log2(adata.shape[0]))), n_pcs=n_pc)
    sc.tl.leiden(adata, resolution=resolution)
    return adata.obs


def get_annotation(gene_id):
    gene_id, _, gene_start, gene_end, strand = get_gene(gene_id)
    tss = gene_start if strand == '+' else gene_end
    gene_cluster = calculate_gene(gene_id)
    this_corr = dmr_corr[dmr_corr.index.get_level_values('Gene') == gene_id]['Corr']
    this_corr.index = this_corr.index.droplevel('Gene')
    gene_cluster['Corr'] = gene_cluster.index.map(this_corr).fillna(0)
    
    this_dmr_bed = dmr_bed.loc[gene_cluster.index]
    dmr_center = (this_dmr_bed['end'] + this_dmr_bed['start']) / 2
    gene_length = gene_end - gene_start
    if strand == '+':
        gene_cluster['reldist_tss'] = (dmr_center - gene_start) / gene_length
    else:
        gene_cluster['reldist_tss'] = (gene_end - dmr_center) / gene_length
    gene_cluster['in_gene_body'] = (gene_cluster['reldist_tss'] > 0) & (gene_cluster['reldist_tss'] < 1)
    
    this_annot = dmr_annot[gene_cluster.index]
    annot_df = pd.DataFrame(this_annot.X.todense(), 
                 index=this_annot.obs_names, columns=this_annot.var_names)
    
    # annotate TE cols
    dna_te = annot_df.columns[20:33]
    gene_cluster['is_dna_te'] = annot_df[dna_te].sum(axis=1) != 0
    
    line_te = annot_df.columns[33:39]
    gene_cluster['is_line_te'] = annot_df[line_te].sum(axis=1) != 0
    
    ltr_te = annot_df.columns[39:45]
    gene_cluster['is_ltr_te'] = annot_df[ltr_te].sum(axis=1) != 0
    
    sine_te = annot_df.columns[45:52]
    gene_cluster['is_sine_te'] = annot_df[sine_te].sum(axis=1) != 0
    
    # this dmr within GOI's gene feature
    gene_cluster['in_intron'] = annot_df['intron'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_exon'] = annot_df['exon'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_utr3'] = annot_df['UTR3'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_utr5'] = annot_df['UTR5'].astype(bool) & gene_cluster['in_gene_body']
    
    # previous mC study
    gene_cluster['feDMR'] = annot_df['feDMR'].astype(bool)
    gene_cluster['adultDMR'] = annot_df['adultDMR'].astype(bool)
    
    
    other_profiles = []
    # DMR hypo call in each cluster
    this_hypo_hits = dmr_hits[gene_cluster.index]
    hits_df = pd.DataFrame(this_hypo_hits.X.todense(), 
                 index=this_hypo_hits.obs_names, columns=this_hypo_hits.var_names)
    hits_df.columns = hits_df.columns.map(lambda i: f'HypoDMR.{i}')
    other_profiles.append(hits_df)
    
    # DMR rate
    related_dmr_rate = dmr_rate.loc[gene_cluster['leiden'].sort_values().index, use_clusters].copy()
    related_dmr_rate.fillna(related_dmr_rate.mean(), axis=0, inplace=True)
    related_dmr_rate.columns = related_dmr_rate.columns.map(lambda i: f'DMRRate.{i}')
    other_profiles.append(related_dmr_rate)
    
    # atac peak
    atac_peak_df = atac_peak.loc[related_dmr_rate.index, use_clusters].copy()
    other_profiles.append(atac_peak_df)
    atac_peak_df.columns = atac_peak_df.columns.map(lambda i: f'ATACPeak.{i}')
    
    dmr_annotation = pd.concat([gene_cluster] + other_profiles, axis=1, sort=True)
    return dmr_annotation

In [15]:
for gene in genes:
    print(gene)
    check_path = output_dir / f'{gene}.DMR_cluster.msg'
    if check_path.exists():
        continue
    
    dmr_annotation = get_annotation(gene)
    dmr_annotation.to_msgpack(output_dir / f'{gene}.DMR_detail.msg', compress='zlib')
    
    cluster_annotation = dmr_annotation.groupby('leiden').mean()
    cluster_annotation.to_msgpack(output_dir / f'{gene}.DMR_cluster.msg', compress='zlib')

ENSMUSG00000033768.17
ENSMUSG00000024642.17
ENSMUSG00000025020.11
ENSMUSG00000025089.15
ENSMUSG00000024897.9
ENSMUSG00000025085.16
ENSMUSG00000039901.10
ENSMUSG00000040929.17
ENSMUSG00000041362.17
ENSMUSG00000048720.8
ENSMUSG00000117897.1
ENSMUSG00000024713.16
ENSMUSG00000024867.14
ENSMUSG00000041731.13
ENSMUSG00000052942.13
ENSMUSG00000024743.16
ENSMUSG00000006435.16
ENSMUSG00000033610.16
ENSMUSG00000024798.15
ENSMUSG00000058624.13
ENSMUSG00000071604.5
ENSMUSG00000039982.8
ENSMUSG00000097930.2
ENSMUSG00000025092.7
ENSMUSG00000074746.3
ENSMUSG00000024921.17
ENSMUSG00000036278.7
ENSMUSG00000024816.12
ENSMUSG00000053617.12
ENSMUSG00000025025.14
ENSMUSG00000025189.9
ENSMUSG00000074733.14
ENSMUSG00000039126.10
ENSMUSG00000033053.4
ENSMUSG00000024754.13
ENSMUSG00000009378.4
ENSMUSG00000024781.16
ENSMUSG00000025026.15
ENSMUSG00000025090.9
ENSMUSG00000024812.11
ENSMUSG00000032946.16
ENSMUSG00000024805.16
ENSMUSG00000036192.15
ENSMUSG00000024924.14
ENSMUSG00000024983.10
ENSMUSG00000035171.9
EN